In [1]:
!pip install openpyxl
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch as t

In [4]:
train_data = pd.read_excel("/content/sample_data/train_dataset.xlsx")
test_data = pd.read_excel("/content/sample_data/test_dataset.xlsx")

# Modeling

---

Подготовим модель, подберем параметры обучения и оценим модель на тестовой выборке

In [6]:
from dataclasses import dataclass

@dataclass
class Config:
    model_name = "cointegrated/rubert-tiny-sentiment-balanced"
    new_model = "seq-cls-ft-system"
    wb_token = 'bab00ed6b8ec6a868aef6917554e2eee8a723676'

config = Config()

In [58]:
import wandb

# Зайдем в W&B
wandb.finish()
wandb.login(key=config.wb_token)

run = wandb.init(
    project='Fine-tune Pre-Trained SEQ-CLS',
    job_type="training"
)

eval/loss,██▇▆▆▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂
eval/recall,▁▁▂▂▄▅▇▇██▇█████████████████████████████
eval/runtime,▁█▁▁▁▂▆▁▆▁▂▁▂▆▂▁▂▂▆▃▁▁▂▂█▁▂▁▁▁▁▅▁▂▁▇▁▂▂▁
eval/samples_per_second,█▁███▆▂█▃█▇█▆▂▇▇▇▇▂▆█▇▆▇▁▇▆██▇▇▃▇▆█▂▇▇▆█
eval/steps_per_second,█▁███▆▂█▃█▇█▆▂▇▇▇▇▂▆█▇▆▇▁▇▆██▇▇▃▇▆█▂▇▇▆█
train/epoch,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇█
train/global_step,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/grad_norm,█▅▆▅▆▄▃▃▂▁▂▁▅▂▄▃▄▂▂▁▂▃▂▃▄▃▃▃▃▃▃▃▂▁▄▅▃▂▄▃
train/learning_rate,▁▁▃▃▃▅▆▆▆▇█████▇▇▇▇▇▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
train/loss,██▇▇▇▆▆▆▅▄▃▃▃▂▂▃▂▃▂▃▃▁▃▂▃▁▃▂▂▂▂▁▁▁▂▂▂▁▁▂
eval/loss,0.60826


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [59]:
from torch import nn

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Модель
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
model = AutoModelForSequenceClassification.from_pretrained(config.model_name)

# Заменяем последний слой и остальные замораживаем
for name, param in model.named_parameters():
    if "bert.encoder.layer.1" in name:
        break
    else:
        param.requires_grad = False

nn.init.xavier_uniform_(model.classifier.weight)
nn.init.zeros_(model.classifier.bias);

In [60]:
from datasets import Dataset
from tqdm.notebook import tqdm


def preprocess_function(examples):
    inputs = tokenizer(
        examples['MessageText'],
        truncation=True,
        padding='max_length',
        max_length=256,
        return_tensors="pt"
    )

    return inputs

train_dataset = Dataset.from_pandas(train_data).map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)

Map:   0%|          | 0/2292 [00:00<?, ? examples/s]

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

In [61]:
from transformers import (Trainer, TrainingArguments,
                          EarlyStoppingCallback, AdamW)

from sklearn.metrics import recall_score

# Зададим метрику
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    score = recall_score(labels, predictions, average="macro")

    return {"eval_recall": score}

In [62]:
# Зададим параметры тренера
model.to('cuda')
model.train()

training_args = TrainingArguments(
    output_dir=config.new_model,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=10,
    gradient_accumulation_steps=2,
    dataloader_num_workers=2,
    num_train_epochs=50,
    weight_decay=0.01,
    learning_rate=1e-5,
    optim="adamw_torch",
    lr_scheduler_type='cosine',
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_recall",
    greater_is_better=True,
    eval_steps=1,
    logging_steps=1,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=training_args,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15, early_stopping_threshold=0.001)]
)

In [63]:
trainer.train()

Epoch,Training Loss,Validation Loss,Recall
1,1.633000,1.628570,0.105992
2,1.487700,1.568177,0.105992
3,1.397400,1.471807,0.105992
4,1.390600,1.353008,0.114430
5,1.192400,1.216831,0.185963
6,1.159100,1.084422,0.263038
7,1.003200,0.956895,0.499381
8,0.943200,0.836279,0.593136
9,0.875700,0.735678,0.613136
10,0.704400,0.661481,0.677806


TrainOutput(global_step=1800, training_loss=0.7016164403822687, metrics={'train_runtime': 257.3081, 'train_samples_per_second': 445.38, 'train_steps_per_second': 6.996, 'total_flos': 422597155737600.0, 'train_loss': 0.7016164403822687, 'epoch': 50.0})

# Оценим модель

---

Импортируем два файла с тестовыми данными и посмотрим на оценки метрик

In [64]:
from torch.nn import functional as f
from sklearn.metrics import classification_report

pred = np.empty(test_data.shape[0])

model.eval()
model.to('cpu')
for i in range(test_data.shape[0]):
    text = test_data.iloc[i]['MessageText']
    inputs = tokenizer(text, truncation=True, max_length=256, return_tensors='pt')
    with t.no_grad():
        logits = model(**inputs).logits
    probs = f.softmax(logits, dim=-1)
    pred[i] = t.argmax(probs, dim=-1).item()

print(classification_report(test_data.labels.astype(int), pred), end='\n\n\n')

              precision    recall  f1-score   support

           0       0.71      0.73      0.72        30
           1       0.59      0.76      0.67        25
           2       0.94      0.85      0.89        79

    accuracy                           0.81       134
   macro avg       0.75      0.78      0.76       134
weighted avg       0.83      0.81      0.81       134





In [65]:
# Сохраним модель
model.save_pretrained('/content/model')
tokenizer.save_pretrained('/content/tokenizer')

('/content/tokenizer/tokenizer_config.json',
 '/content/tokenizer/special_tokens_map.json',
 '/content/tokenizer/vocab.txt',
 '/content/tokenizer/added_tokens.json',
 '/content/tokenizer/tokenizer.json')